In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

macro = pd.read_csv('macro.csv')
dd = pd.read_csv('data_dict.csv')
train = pd.read_csv('train.csv')

In [2]:
train.columns

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price_doc'],
      dtype='object', length=292)

## Picking Features from Macro

In [3]:
macro.head()
m = macro.drop(['timestamp', 'child_on_acc_pre_school', 'modern_education_share', 'old_education_build_share'], axis=1)
m = m.dropna()

In [4]:
data_scaled = pd.DataFrame(preprocessing.scale(m), columns = m.columns) 

pca = PCA(n_components=1)
pca.fit_transform(data_scaled)

results = pd.DataFrame(pca.components_,columns=data_scaled.columns)
results = results.abs()
results.sort_values(by = [0], axis=1, ascending=False, inplace=True)

/Users/brookelhanna/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:197: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [5]:
results

,mortgage_value,deposits_value,mortgage_rate,net_capital_export,rent_price_2room_eco,rts,rent_price_1room_eco,micex,gdp_quart,rent_price_1room_bus,...,pop_total_inc,pop_natural_increase,unprofitable_enterpr_share,invest_fixed_assets_phys,divorce_rate,marriages_per_1000_cap,fin_res_per_cap,overdue_wages_per_cap,share_own_revenues,apartment_fund_sqm
0,0.246347,0.243305,0.238557,0.232085,0.231692,0.229822,0.228511,0.227866,0.225961,0.212284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
macro_cols = ['mortgage_value', 'deposits_value', 'mortgage_rate', 'net_capital_export', 'rent_price_2room_eco', 
              'rts','rent_price_1room_eco', 'micex', 'gdp_quart', 'rent_price_1room_bus']
macrodf = macro[['timestamp'] + macro_cols]
len(macro_cols)

10

## Picking Features from Dataset

In [7]:
train2 = train.drop(columns=['timestamp', 'culture_objects_top_25', 'product_type', 'sub_area', 'nuclear_reactor_raion',
                             'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 
                            'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'ecology',
                             'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line'])
train2 = train2.dropna()
train_scaled = pd.DataFrame(preprocessing.scale(train2), columns = train2.columns) 

pca2 = PCA(n_components = 1)
pca2.fit_transform(train_scaled)

train_results = pd.DataFrame(pca2.components_, columns = train_scaled.columns)
train_results = train_results.abs()
train_results.sort_values(by = [0], axis=1, ascending=False, inplace=True)

/Users/brookelhanna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  


In [8]:
''' Picked these columns from the highest PCA scores, but also got rid of duplicates 
like office_count_3000 and office_count_5000'''

train_cols = ['office_count_3000','cafe_count_3000','church_count_3000','leisure_count_3000','big_church_count_3000',
              'office_count_5000','office_sqm_3000','sport_count_3000','trc_count_3000','kremlin_km','mkad_km',
              'sport_objects_raion', 'bulvar_ring_km','build_count_brick', 'university_top_20_raion','sadovoe_km',
              'mosque_count_3000','zd_vokzaly_avto_km','shopping_centers_raion','school_education_centers_top_20_raion',
              'build_count_1921-1945','green_part_5000','additional_education_raion','ID_metro','museum_km','theater_km',
              'work_all','detention_facility_km','power_transmission_line_km','children_preschool', 'ice_rink_km',
             'industrial_km', 'kindergarten_km','university_km','green_zone_part','0_17_male','ekder_female','0_17_all',
             'hospice_morgue_km','basketball_km']
len(train_cols)

40

In [9]:
traindf = train[['id', 'timestamp', 'price_doc'] + train_cols]

In [10]:
df = traindf.merge(macrodf, on='timestamp')

### Filling in Missing Values

In [11]:
df[['build_count_brick', 'build_count_1921-1945']].head()

,build_count_brick,build_count_1921-1945
0,0.0,0.0
1,67.0,1.0
2,206.0,0.0
3,124.0,24.0
4,643.0,114.0


In [12]:
(df[['build_count_brick']].mean(), df[['build_count_1921-1945']].mean())

(build_count_brick    108.048705
 dtype: float64, build_count_1921-1945    26.721586
 dtype: float64)

In [13]:
df[['build_count_brick']] = df[['build_count_brick']].fillna(108.0)
df[['build_count_1921-1945']] = df[['build_count_1921-1945']].fillna(26.7)

In [14]:
df.to_csv("house_dataset.csv", index=False)

## Data Dictionary

In [15]:
data = pd.read_csv('house_dataset.csv')
dd = pd.read_csv('data_dict.csv')

In [16]:
ids = []
for index, row in dd.iterrows():
    if row['column'] in list(data.columns):
        ids.append(index)

In [17]:
data_dictionary = dd.iloc[ids]
# remove duplicate
data_dictionary.drop([38], axis=0, inplace = True)

/Users/brookelhanna/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
data_dictionary.to_csv("features_dict.csv", index = False)

## Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression
y = df[['price_doc']]
x = df.drop(columns = ['id', 'timestamp', 'price_doc'])
reg = LinearRegression().fit(x, y)
reg.score(x,y)

0.19315113239466353

In [20]:
x = df.drop(columns = ['id', 'timestamp'])
y = train[['price_doc']]
reg = LinearRegression()

In [21]:
# cross val
cross_val_score(reg, x, y, cv=5, scoring='neg_mean_squared_error').mean()

-9.891878563412996e-18

In [22]:
# R-Squared
reg.fit(x,y)
reg.score(x,y)

1.0

In [23]:
# get coefficients for report
c1=np.transpose(pd.DataFrame(reg.coef_))[:25]
c2=np.transpose(pd.DataFrame(reg.coef_))[25:]
pd.DataFrame({'coefficients': pd.Series(list(c1[0])), 'coefficients_2': pd.Series(list(c2[0]))}).to_csv("weights.csv", index=False)